## 목차
### 1. Library Import
### 2. 학습데이터 생성
### 3. Light-gbm 모델 훈련
### 4. 교차 검증 점수 확인
### 5. 제출 파일 생성

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Library Import

In [2]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=ab86fc08dcbc53ef0a1280e030694283073c2b5d065345b407fc059a1fc2b1da
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc #garbage collection 감시
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
import seaborn as sns
from functools import partial
from bayes_opt import BayesianOptimization
from lightgbm import LGBMClassifier

warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    min  = int(x[10:12])
    sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)


PATH = '/content/drive/MyDrive/YBIGTA/21-1 겨울방학/lg 품질변화 공모전/quality_change_prediction/data/'

# 2. 학습 데이터 생성

## 2.1 train_err

* train_err_data.csv 파일의 errtype column의 value_counts를 카운트하여 학습에 사용.  
* 주어진 기간동안 user_id별로 peroblem가 한 번이라도 발생했는 지를 확인.  

In [ ]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
display(train_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [ ]:
train_err["model_nm"].unique()

array(['model_3', 'model_2', 'model_0', 'model_1', 'model_7', 'model_4',
       'model_5', 'model_8', 'model_6'], dtype=object)

In [ ]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
display(train_err.head())
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [ ]:
print(np.sort(train_err.errtype.unique()))
# errtype이 1부터 42까지 29를 제외한 41개가 존재. 
#이후 나올 데이터에서는 29 또는 42 이후 error 존재할 확률

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24

 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [ ]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

#사람별로 어떤 error가 몇번 일어났는지
for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████| 16554663/16554663 [00:40<00:00, 411976.72it/s]


(15000, 42)

In [ ]:
#참고) 일별 합산 데이터 생성 코드      
'''
train_err             = pd.read_csv(PATH+'train_err_data.csv')
train_err['datetime'] = train_err['time'].apply(make_datetime)

min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

id_error = train_err[['user_id','errtype','days']].values
error = np.zeros((train_user_number,42, 33))
for idx, err, days in tqdm(id_error):
    error[idx - train_user_id_min,err - 1, days] += 1
error.shape
'''

"\ntrain_err             = pd.read_csv(PATH+'train_err_data.csv')\ntrain_err['datetime'] = train_err['time'].apply(make_datetime)\n\nmin_day = train_err.datetime.min().date()\n\ntrain_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day\ntrain_err['days'] = train_err['days'].dt.days\ndisplay(train_err)\n\nid_error = train_err[['user_id','errtype','days']].values\nerror = np.zeros((train_user_number,42, 33))\nfor idx, err, days in tqdm(id_error):\n    error[idx - train_user_id_min,err - 1, days] += 1\nerror.shape\n"

## 2.2 problem

In [ ]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

## EDA

# 3. Light-gbm 모델 훈련

In [16]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train = pd.read_csv("/content/drive/MyDrive/YBIGTA/21-1 겨울방학/Jupyter Temp/lg 공모전/data/0129_train.csv")
test = pd.read_csv("/content/drive/MyDrive/YBIGTA/21-1 겨울방학/Jupyter Temp/lg 공모전/data/0129_test.csv")


In [17]:
train.head()

,Unnamed: 0,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,fw_rank,q_meanmax_0,q_meanmax_1,q_meanmax_2,q_meanmax_5,q_meanmax_6,q_meanmax_11,q_chg_0,q_chg_1,q_chg_2,q_chg_5,q_chg_6,q_chg_11,weekend_04,weekend_28,weekend_29,weekend_35
0,0,10000,1,0,0.0,0.0,4.969736,224.193652,0.000000,3.009071,3.307497,3.158235,14.282485,19.048874,3.722281,34.004624,45.551438,56.487550,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,23.310719,0.083029,0.086005,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.833333,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.0,0.0,0.0,2.0,0.0,0.0,120.0,28.0,0.0,0.0
1,1,10001,1,1,0.0,0.0,0.000000,0.000000,20.829435,3.009071,3.307497,0.000000,9.521657,13.096101,3.722281,51.006936,116.580799,118.531251,0.000000,14.307295,5.755239,8.267622,6.295681,639.260538,680.131311,5.13495,7.2621,16.026120,0.000000,0.000000,0.0,4158.947266,0.0,7.444794,19.655867,0.0,5.53321,0.307621,0.000000,0.0,197.897989,59.040125,0.682842,0.967949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,10002,1,0,0.0,0.0,1.242434,284.553481,0.393008,6.018142,3.307497,0.451176,12.378154,16.667765,3.722281,13.601849,40.147030,48.153321,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.05398,0.0000,18.211500,0.166058,0.172010,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.833333,1.666667,0.166667,1.666667,0.666667,3.666667,0.166667,2.0,1.0,2.0,12.0,3.0,1.0,216.0,0.0,0.0,0.0
3,3,10003,1,1,0.0,0.0,0.000000,0.000000,0.786016,3.009071,3.307497,0.000000,8.569491,10.714992,0.000000,0.000000,40.147030,27.780762,0.934418,0.000000,0.000000,0.000000,0.000000,23.676316,17.207050,0.00000,0.0000,42.979139,0.000000,0.000000,0.0,1081.326289,0.0,5.955835,0.000000,0.0,5.53321,0.307621,0.613991,0.0,29.772264,1.054288,0.000000,0.947059,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,10004,1,1,0.0,0.0,0.000000,2.155708,0.000000,9.027214,13.229988,0.000000,15.234651,22.620538,11.166842,17.002312,110.404333,84.268312,0.000000,0.000000,0.000000,0.000000,0.000000,118.381581,107.770474,0.00000,0.0000,24.039179,0.000000,0.000000,0.0,2927.898875,0.0,11.911670,0.000000,0.0,5.53321,0.307621,0.000000,0.0,7.005239,0.000000,1.365683,0.833977,1.666667,0.166667,1.666667,0.416667,7.250000,0.166667,1.0,1.0,1.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0


In [18]:
train.columns

Index(['Unnamed: 0', 'user_id', 'model_nm', 'update', 'err_1', 'err_2',
       'err_3', 'err_4', 'err_5', 'err_6', 'err_7', 'err_10', 'err_11',
       'err_12', 'err_13', 'err_14', 'err_15', 'err_16', 'err_17', 'err_18',
       'err_19', 'err_20', 'err_21', 'err_22', 'err_23', 'err_24', 'err_25',
       'err_26', 'err_27', 'err_28', 'err_30', 'err_31', 'err_32', 'err_33',
       'err_34', 'err_35', 'err_36', 'err_37', 'err_38', 'err_39', 'err_40',
       'err_41', 'err_42', 'fw_rank', 'q_meanmax_0', 'q_meanmax_1',
       'q_meanmax_2', 'q_meanmax_5', 'q_meanmax_6', 'q_meanmax_11', 'q_chg_0',
       'q_chg_1', 'q_chg_2', 'q_chg_5', 'q_chg_6', 'q_chg_11', 'weekend_04',
       'weekend_28', 'weekend_29', 'weekend_35'],
      dtype='object')

In [31]:
for i in train.columns:
  print(i,train[i].nunique())

Unnamed: 0 15000
user_id 15000
model_nm 3
update 3
err_1 81
err_2 63
err_3 86
err_4 593
err_5 829
err_6 150
err_7 139
err_10 166
err_11 199
err_12 213
err_13 61
err_14 277
err_15 409
err_16 383
err_17 105
err_18 16
err_19 13
err_20 14
err_21 13
err_22 997
err_23 1134
err_24 149
err_25 23
err_26 154
err_27 442
err_28 440
err_30 4
err_31 589
err_32 895
err_33 110
err_34 280
err_35 69
err_36 6
err_37 6
err_38 32
err_39 101
err_40 456
err_41 150
err_42 20
fw_rank 9099
q_meanmax_0 76
q_meanmax_1 41
q_meanmax_2 89
q_meanmax_5 1340
q_meanmax_6 568
q_meanmax_11 20
q_chg_0 105
q_chg_1 114
q_chg_2 110
q_chg_5 341
q_chg_6 127
q_chg_11 114
weekend_04 270
weekend_28 233
weekend_29 1
weekend_35 42


In [28]:
cor = train.corr()
for i in train.columns:
  for j in train.columns:
    if cor.loc[i,j] >0.7 and i != j:
      print("변수1: ",i,"  / 변수2: ", j,"   / 상관계수: ", train.corr().loc[i,j])

변수1:  Unnamed: 0   / 변수2:  user_id    / 상관계수:  1.0
변수1:  user_id   / 변수2:  Unnamed: 0    / 상관계수:  1.0
변수1:  update   / 변수2:  err_36    / 상관계수:  0.8817461103916505
변수1:  update   / 변수2:  err_37    / 상관계수:  0.8782740282806293
변수1:  err_3   / 변수2:  err_11    / 상관계수:  0.7469859466154408
변수1:  err_3   / 변수2:  err_12    / 상관계수:  0.7404118920090496
변수1:  err_4   / 변수2:  weekend_04    / 상관계수:  0.872372959060474
변수1:  err_6   / 변수2:  err_7    / 상관계수:  0.9104036576216407
변수1:  err_7   / 변수2:  err_6    / 상관계수:  0.9104036576216407
변수1:  err_7   / 변수2:  err_35    / 상관계수:  0.7723774734313907
변수1:  err_11   / 변수2:  err_3    / 상관계수:  0.7469859466154408
변수1:  err_11   / 변수2:  err_12    / 상관계수:  0.9917618839187488
변수1:  err_12   / 변수2:  err_3    / 상관계수:  0.7404118920090496
변수1:  err_12   / 변수2:  err_11    / 상관계수:  0.9917618839187488
변수1:  err_15   / 변수2:  err_16    / 상관계수:  0.95965450843297
변수1:  err_16   / 변수2:  err_15    / 상관계수:  0.95965450843297
변수1:  err_18   / 변수2:  err_20    / 상관계수:  0.78411086315

In [11]:
test.head()

,Unnamed: 0,user_id,model_nm,update,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,fw_rank,q_meanmax_0,q_meanmax_1,q_meanmax_2,q_meanmax_5,q_meanmax_6,q_meanmax_11,q_chg_0,q_chg_1,q_chg_2,q_chg_5,q_chg_6,q_chg_11,weekend_04,weekend_28,weekend_29,weekend_35
0,0,30000,2.0,1.0,0.0,0.0,0.000000,0.000000,24.366509,3.009071,3.307497,0.0,15.234651,19.048874,0.000000,10.201387,51.727904,34.262940,8.409761,10.730471,0.0,2.755874,0.0,831.207816,1184.569580,5.13495,14.5242,23.310719,0.000000,0.000000,0.0,1380.770492,2.139521,11.911670,0.000000,3.422427,16.599629,0.922864,0.000000,3.788643,161.120487,0.000000,1.365683,0.712195,0.000000,0.000000,0.000000,0.166667,0.416667,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,35.0
1,1,30001,1.0,0.0,0.0,0.0,1.863651,211.259403,1.179025,12.036285,13.229988,0.0,11.425988,19.048874,14.889123,20.402774,40.919088,50.931397,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00000,0.0000,17.483040,0.083029,0.086005,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,7.500000,0.750000,7.500000,213.000000,0.750000,0.750000,1.0,1.0,1.0,7.0,1.0,1.0,120.0,0.0,0.0,0.0
2,2,30002,1.0,1.0,0.0,0.0,0.000000,0.000000,31.047648,6.018142,3.307497,0.0,15.234651,19.048874,0.000000,3.400462,133.566081,123.161378,0.000000,0.000000,0.0,0.000000,0.0,50.734963,36.225369,2.05398,0.0000,29.138399,0.000000,0.000000,0.0,3826.231484,0.000000,11.911670,0.000000,0.000000,5.533210,0.307621,0.000000,26.520504,197.897989,1.054288,1.365683,0.676553,4.166667,0.416667,4.166667,4.083333,0.416667,0.416667,3.0,3.0,3.0,16.0,3.0,3.0,0.0,0.0,0.0,0.0
3,3,30003,1.0,1.0,0.0,0.0,0.000000,0.000000,42.051878,6.018142,6.614994,0.0,3.808663,5.952773,3.722281,23.803237,6.176466,11.112305,0.000000,3.576824,0.0,2.755874,0.0,2.536748,1.811268,0.00000,0.0000,3.642300,0.166058,0.172010,0.0,2229.195734,0.000000,2.977918,17.471882,10.267280,5.533210,0.307621,0.000000,0.000000,70.052385,8.434304,0.000000,0.697623,0.000000,0.000000,0.000000,0.500000,7.750000,0.000000,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0
4,4,30004,2.0,1.0,0.0,0.0,0.000000,4.311416,2.358049,3.009071,3.307497,0.0,9.521657,14.286655,3.722281,61.208323,61.764662,62.043702,5.606507,0.000000,0.0,0.000000,0.0,8.455827,7.245074,0.00000,0.0000,10.198440,0.083029,0.086005,0.0,5523.081969,162.603583,8.189273,6.551956,3.422427,5.533210,0.307621,0.306995,0.000000,84.062863,94.885916,0.000000,0.662306,5.000000,0.500000,5.000000,0.500000,2.500000,0.500000,1.0,1.0,1.0,2.0,3.0,1.0,8.0,28.0,0.0,35.0


In [15]:
train_x = train.drop(columns="label")
train_y = train["label"]

test_x = test

KeyError: ignored

In [ ]:
def lgbm_cv(colsample_bytree, 
            learning_rate,
            max_depth,
            n_estimators,
            num_leaves,
          x_data=None, y_data=None, n_splits=5, output='score'):
    
    score = 0
    kf = KFold(n_splits=n_splits, random_state=4321)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data[train_index], y_data[train_index]
        x_valid, y_valid = x_data[valid_index], y_data[valid_index]

        model = LGBMClassifier(
            colsample_bytree = colsample_bytree,
            learning_rate = learning_rate,
            max_depth = int(max_depth),
            n_estimators = int(n_estimators),
            num_leaves = int(num_leaves)                        
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        prob = model.predict(x_valid)
        pred = np.where(prob > 0.6, 1, 0)
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
func_fixed = partial(lgbm_cv, x_data=train_x, y_data=train_y, n_splits=5, output="score")
regBO = BayesianOptimization(
    func_fixed, 
    {
         'colsample_bytree' : (0.5,1),
         'learning_rate':(0.01, 0.1),
         "max_depth" : (200,700),
         "n_estimators" : (30,50),
         "num_leaves" : (200,600),
    }, random_state=1029)

regBO.maximize(init_points=100, n_iter=0)

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | num_le... |

-------------------------------------------------------------------------------------

|  1        |  0.6975   |  0.5205   |  0.04688  |  494.4    |  48.0     |  248.4    |

|  2        |  0.7042   |  0.5851   |  0.09462  |  497.7    |  36.71    |  376.7    |

|  3        |  0.6642   |  0.7479   |  0.01791  |  337.1    |  43.2     |  232.1    |

|  4        |  0.7017   |  0.5193   |  0.07076  |  293.7    |  48.79    |  548.5    |

|  5        |  0.7011   |  0.6791   |  0.06111  |  639.6    |  39.05    |  482.0    |

|  6        |  0.705    |  0.8981   |  0.09494  |  351.9    |  48.19    |  400.0    |

|  7        |  0.6774   |  0.5194   |  0.03405  |  258.2    |  35.68    |  396.2    |

|  8        |  0.7066   |  0.8786   |  0.07362  |  464.0    |  47.4     |  209.2    |

|  9        |  0.6792   |  0.6541   |  0.02515  |  413.0    |  45.54    |  413.7    |

|  10       |  0.6955   |  0.6458   |  0.05

In [ ]:
params = regBO.max['params']

In [ ]:
params

{'colsample_bytree': 0.9532847935680113,
 'learning_rate': 0.09278453605939078,
 'max_depth': 578.1033906895178,
 'n_estimators': 39.602849442683194,
 'num_leaves': 413.07521762936597}

In [ ]:
models = lgbm_cv(
    params['colsample_bytree'], 
    params['learning_rate'],
    params["max_depth"],
    params["n_estimators"],
    params["num_leaves"],
    x_data=train_x, y_data=train_y, n_splits=5, output='model')

# 5. 제출 파일 생성

In [ ]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [ ]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [ ]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532648/16532648 [00:40<00:00, 404165.80it/s]

(14999, 42)


In [ ]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [ ]:
pred_ensemble

array([[1. ],
       [0. ],
       [0. ],
       ...,
       [0.6],
       [1. ],
       [0.2]])

In [ ]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [ ]:
SUB_PATH = '/content/drive/MyDrive/YBIGTA/21-1 겨울방학/lg 품질변화 공모전/quality_change_prediction/submissions/'
sample_submssion.to_csv(SUB_PATH+"dacon_baseline_0111(3).csv", index = False)
sample_submssion

,user_id,problem
0,30000,1.0
1,30001,0.0
2,30002,0.0
3,30003,1.0
4,30004,1.0
...,...,...
14994,44994,0.0
14995,44995,0.0
14996,44996,0.6
14997,44997,1.0
